# Design Patterns 2

## Bridge

Angenommen, wir wollen das Maze aus dem vorherigen Kapitel nun sowohl mit Java Swing als auch für Android implementieren. Hierzu muss die Funktionalität der `draw` Methode der `MapSite`s jeweils für Android und Swing implementiert werden, und das für jede Art von Maze-Komponente. Dies kann zu einer Grossen Anzahl an Unterklassen führen. Eine Alternative bietet das Bridge-Pattern: Die Maze-Komponenten interagieren mit einem abstrakten Implementor-Interface, das den Client von der konkreten Umsetzung (Android vs Swing) abstrahiert. 

In [1]:
interface MapRenderer {
    void updateGUI(String element);
}

Von diesem Implementor-Interface gibt es nun verschiedene konkrete Umsetzungen.

In [2]:
class PrintRenderer implements MapRenderer {
    public void updateGUI(String element) {
        System.out.println(element);
    }
}

In [3]:
class SwingRenderer implements MapRenderer {
    public void updateGUI(String element) {
        // Code to handle Java Swing output
    }
}

In [4]:
class AndroidRenderer implements MapRenderer {
    public void updateGUI(String element) {
        // Code to handle Android output
    }
}

Statt für jede Art von Output eine eigene Unterklasse von `MapSite` zu definieren, bekommt die `MapSite` eine Referenz auf ein Implementor-Objekt, und kann damit unabhängig von der Implementierung der GUI umgesetzt werden.

In [5]:
public abstract class MapSite {
    protected MapRenderer impl;
    
    public MapSite(MapRenderer renderer) {
        this.impl = renderer;
    }

    public MapSite() {
        this(new PrintRenderer());
    }

    public abstract void draw();
}

Die konkreten Komponenten bedienen nun das `impl` Objekt.

In [6]:
class Door extends MapSite {

    public Door() {
        super();
    }

    public Door(MapRenderer renderer) {
        super(renderer);
    }

    @Override
    public void draw() {
        impl.updateGUI("Door");
    }
}

In [7]:
class Wall extends MapSite {
    @Override
    public void draw() {
        impl.updateGUI("Wall");
    }
}

In [8]:
public abstract class Composite extends MapSite {
    protected List<MapSite> elements = new ArrayList<>();

    public void add(MapSite x) {
        elements.add(x);
    }
    
    public List<MapSite> getChildren() {
        return elements;
    }
  
    @Override
    public void draw() {
        elements.forEach(e -> e.draw());
    }
}

In [9]:
class Room extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Room consisting of:");
        elements.forEach(x -> x.draw());
    }
}

In [10]:
class Maze extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Maze consisting of: ");
        elements.forEach(x -> x.draw());
    }
}

Nachdem wir `MapSite` so implementiert haben, dass per Default ein `PrintRenderer` verwendet wird, können wir wie gehabt unser Maze erzeugen.

In [11]:
Maze maze = new Maze();
Room room1 = new Room();
Room room2 = new Room();
room1.add(new Wall());
room1.add(new Wall());
room1.add(new Wall());
room1.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room1.add(new Door());
room2.add(new Door());

maze.add(room1);
maze.add(room2);

maze.draw();

Maze consisting of: 
Room consisting of:
Wall
Wall
Wall
Wall
Door
Room consisting of:
Wall
Wall
Wall
Wall
Door


## Facade

Wir haben im obigen Beispiel keine tatsächliche Implementierung von `SwingRenderer` angegeben, und das mit gutem Grund: Um eine Swing-GUI zu verwenden muss eine Vielzahl an Klassen konfiguriert und verwendet werden. Das Facade-Entwurfsmuster versteckt diese Komplexität hinter einer Fassade, die ein einfacheres, abstrakteres Interface zur Verfügrung stellt.

In [12]:
class SwingFacade {
    public void drawWindow() {
        // ...
        // Actual Swing code
        System.out.println("...simulating Swing code...");
    }
    
    public void renderText(String text) {
        // ...
        // Actual Swing code
        System.out.println("...simulating Swing code...");
    }
    
    public void renderButtons() {
        // ...
        // Actual Swing code
        System.out.println("...simulating Swing code...");
    }
}

In [13]:
class SwingRenderer implements MapRenderer {
    private SwingFacade facade = new SwingFacade();
    
    public void updateGUI(String element) {
        facade.drawWindow();
        facade.renderText(element);
        facade.renderButtons();
    }
}

In [14]:
Door doorUsingFacade = new Door(new SwingRenderer());

In [15]:
doorUsingFacade.draw()

...simulating Swing code...
...simulating Swing code...
...simulating Swing code...


## Adapter

Angenommen es gibt bereits eine bestehende Komponente `WebRenderer`, die zwar genau die Funktionalität bietet, die wir in unserem Beispiel von Unterklassen des `MapRenderer` Interfaces erwarten, aber die Klasse `WebRenderer` implementiert dieses Interface nicht und hat anders benannte Methoden. Beispielsweise könnte diese Klasse aus einer verwendeten Bibliothek stammen.

In [16]:
class WebRenderer {
    void renderStuffOnTheWeb(String element) {
        System.out.println("Web rendering of " + element);
    }
}

Das Adapter-Entwurfsmuster erlaubt uns, so eine Komponente in unsere Interfaces einzupassen.

In [17]:
class WebRendererAdapter implements MapRenderer {
    private WebRenderer adaptee = new WebRenderer();

    @Override
    public void updateGUI(String element) {
        adaptee.renderStuffOnTheWeb(element);
    }
}

In [18]:
Door door = new Door(new WebRendererAdapter());

In [19]:
door.draw();

Web rendering of Door


## Proxy

Ein Proxy ist ein Wrapper-Objekt das vom Client wie das tatsächliche Objekt benutzt wird, und dabei den Zugriff auf das tatsächliche Objekt steuert. Als Beispiel verwenden wir einen Proxy um Lazy Initialisation durchzuführen.

In [20]:
class WebRendererProxy implements MapRenderer {
    private MapRenderer realSubject = null;
    
    private void setupCostlyInitialisation() {
        if (realSubject == null) {
            realSubject = new WebRendererAdapter();
        }
    }
   
    public void updateGUI(String element) {
        setupCostlyInitialisation();
        realSubject.updateGUI(element);
    }
}

Der Proxy ist eine für den Client transparente Schicht zur tatsächlichen Implementierung; wir verwenden diese Schicht für Lazy Initialisation, andere Anwendungsbeispiele sind Remote Proxies, Logging Proxies, oder Zugriffskontrolle.

In [21]:
Door door = new Door(new WebRendererProxy());

In [22]:
door.draw()

Web rendering of Door


## Visitor

Angenommen wir wollen eine komplexe Objektstruktur um Funktionalität erweitern, ohne dabei die neue Funktionalität direkt in den Klassen der Objektstruktur zu implementieren. Beispielsweise könnten wir ein Maze als XML Code ausgeben lassen. Ein einfacher Ansatz wäre, die Funktionalität in eine eigene Klasse zu packen.

In [23]:
class XMLConverter {
    void visitDoor(Door door) {
        // print out xml for door
        System.out.println("Door");
    }
    
    void visitWall(Wall wall) {
        // print out xml for wall
        System.out.println("Wall");
    }

    void visitRoom(Room room) {
        // print out xml for room
        System.out.println("Room");

    }

    void visitMaze(Maze maze) {
        // print out xml for maze
        System.out.println("Maze");
    }  
}

Das Problem dabei ist, wie wir entscheiden welche der Funktionen nun aufgerufen werden soll. Wenn wir in irgendeiner Form über unsere Objektstruktur iterieren, müssen wir dabei bei jedem Objekt aufgrund der Klasse entscheiden welche Funktion die passende ist.

In [24]:
XMLConverter converter = new XMLConverter();

for (MapSite mapSite : Arrays.asList(maze, door)) {
    if (mapSite instanceof Door) {
        converter.visitDoor((Door) mapSite);
    } else if (mapSite instanceof Wall) {
        converter.visitWall((Wall) mapSite);
    } else if (mapSite instanceof Room) {
        converter.visitRoom((Room) mapSite);
    } else if (mapSite instanceof Maze) {
        converter.visitMaze((Maze) mapSite);
    } 
}

Maze
Door


Das ist umständlich, und man fragt sich vielleicht warum man das nicht einfach mit dynamischem Typing löst:

In [25]:
class XMLConverter {
    void visit(Door door) {
        // print out xml for door
        System.out.println("Door");
    }
    
    void visit(Wall wall) {
        // print out xml for wall
        System.out.println("Wall");

    }

    void visit(Room room) {
        // print out xml for room
        System.out.println("Room");

    }

    void visit(Maze maze) {
        // print out xml for maze
        System.out.println("Maze");
    }  

    void visit(MapSite mapsite) {
        // fallback, we don't actually want this to be called
        System.out.println("MapSite");
    }  

}

In [26]:
MapSite mapSite = maze;
new XMLConverter().visit(mapSite)

MapSite


...scheint ja zu funktionieren. Allerdings funktioniert es nicht mehr, wenn der Code die Abstraktion benutzt, so wie es wohl der Fall wäre wenn man über mehrere Elemente iteriert.

In [27]:
XMLConverter converter = new XMLConverter();

for (MapSite mapSite : Arrays.asList(maze, door)) {
    converter.visit(mapSite);
}

MapSite
MapSite


Im Visitor-Pattern wird dieses Problem mit Hilfe von _Double Dispatch_ gelöst. Leider habe ich noch keine gute Möglichkeit gefunden, die Visitor-Klassen/Interfaces in geeigneter Reihenfolge im Jupyter Notebook, sondern muss einen Umweg in den Definition gehen. Diese doppelten Definitionen wären normalerweise nicht notwendig, sondern sind nur ein Problem dass hier im Notebook besteht!

In [28]:
public interface Visitor {
    // Jupyter-Hack: Wird weiter unten vervollständigt
}

Wir müssen unsere Klassenhierarchie um ein Interface erweitern, sodass alle Objekte eine `accept` Methode erhalten, die das Double Dispatch durchführen kann.

In [29]:
public interface Visitable {
    void accept(Visitor v);
}

Eine `MapSite` soll `Visitable` sein.

In [30]:
public abstract class MapSite implements Visitable {
    protected MapRenderer impl;
    
    public MapSite(MapRenderer renderer) {
        this.impl = renderer;
    }

    public MapSite() {
        this(new PrintRenderer());
    }

    public abstract void draw();
    
    public abstract void accept(Visitor v);
}

Nachdem sich das Interface verändert hat, müssen alle Klassen der Objektstruktur so erweitert werden, dass sie auch die neue `accept` Methode implementieren.

An dieser Stelle nun der Jupyter-Hack: Die `accept` Methode bleibt vorerst noch leer, normalerweise würde hier gleich der Code folgen...

In [31]:
class Door extends MapSite {

    @Override
    public void draw() {
        impl.updateGUI("Door");
    }
    
    @Override
    public void accept(Visitor v) {
        // Jupyter-Hack: Wird weiter unten vervollständigt
    }
}

In [32]:
class Wall extends MapSite {
    @Override
    public void draw() {
        impl.updateGUI("Wall");
    }
    
    @Override
    public void accept(Visitor v) {
        // Jupyter-Hack: Wird weiter unten vervollständigt
    }
}

In [33]:
public abstract class Composite extends MapSite {
    protected List<MapSite> elements = new ArrayList<>();

    public void add(MapSite x) {
        elements.add(x);
    }

    public List<MapSite> getChildren() {
        return elements;
    }

    @Override
    public void draw() {
        elements.forEach(e -> e.draw());
    }
}

In [34]:
class Room extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Room consisting of:");
        elements.forEach(x -> x.draw());
    }
    
    @Override
    public void accept(Visitor v) {
        // Jupyter-Hack: Wird weiter unten vervollständigt
        elements.forEach(x -> x.accept(v));
    }
}

In [35]:
class Maze extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Maze consisting of: ");
        elements.forEach(x -> x.draw());
    }
    
    @Override
    public void accept(Visitor v) {
        // Jupyter-Hack: Wird weiter unten vervollständigt
        elements.forEach(x -> x.accept(v));
    }
}

Nachdem wir das gemacht haben, ist nun auch Jupyter bereit die vollständige Definition zu akzeptieren. Unser Visitor benötigt eine eigene Methode für jedes Element der Objektstruktur.

In [36]:
public interface Visitor {
    void visitDoor(Door d);
    void visitRoom(Room r);
    void visitMaze(Maze m);
    void visitWall(Wall w);
}

Die jeweiligen `accept` Methoden der konkreten Klassen der Objektstruktur rufen dann die entsprechende Methode auf (das ist das Double Dispatch).

In [37]:
class Door extends MapSite {

    @Override
    public void draw() {
        impl.updateGUI("Door");
    }
    
    @Override
    public void accept(Visitor v) {
        v.visitDoor(this);
    }
}

In [38]:
class Wall extends MapSite {
    @Override
    public void draw() {
        impl.updateGUI("Wall");
    }
    
    @Override
    public void accept(Visitor v) {
       v.visitWall(this);
    }
}

In [39]:
public abstract class Composite extends MapSite {
    protected List<MapSite> elements = new ArrayList<>();

    public void add(MapSite x) {
        elements.add(x);
    }

    public List<MapSite> getChildren() {
        return elements;
    }

    @Override
    public void draw() {
        elements.forEach(e -> e.draw());
    }
}

In [40]:
class Room extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Room consisting of:");
        elements.forEach(x -> x.draw());
    }
    
    @Override
    public void accept(Visitor v) {
        v.visitRoom(this);
        elements.forEach(x -> x.accept(v));
    }
}

In [41]:
class Maze extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Maze consisting of: ");
        elements.forEach(x -> x.draw());
    }
    
    @Override
    public void accept(Visitor v) {
        v.visitMaze(this);
        elements.forEach(x -> x.accept(v));
    }
}

In [42]:
public class PrintVisitor implements Visitor {
    @Override
    public void visitDoor(Door d) {
        System.out.println("Here's a door");
    }

    @Override
    public void visitWall(Wall w) {
        System.out.println("Here's a wall");
    }

    @Override
    public void visitRoom(Room r) {
        System.out.println("Here's a room");
    }

    @Override
    public void visitMaze(Maze m) {
        System.out.println("Here's a maze");
    }

}

In [43]:
Maze maze = new Maze();
Room room1 = new Room();
Room room2 = new Room();
room1.add(new Wall());
room1.add(new Wall());
room1.add(new Wall());
room1.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room1.add(new Door());
room2.add(new Door());

maze.add(room1);
maze.add(room2);

In [44]:
PrintVisitor visitor = new PrintVisitor()

In [45]:
room1.accept(visitor)

Here's a room
Here's a wall
Here's a wall
Here's a wall
Here's a wall
Here's a door


In [46]:
maze.accept(visitor)

Here's a maze
Here's a room
Here's a wall
Here's a wall
Here's a wall
Here's a wall
Here's a door
Here's a room
Here's a wall
Here's a wall
Here's a wall
Here's a wall
Here's a door


In dieser Implementierung haben wir die Traversierung der Objektstruktur in der Objektstruktur selbst umgesetzt; d.h. Composite Objekte rufen innerhalb der `accept` Methode die `accept` Methode ihrer Kinder auf. Es ist auch Möglich dies im Visitor selbst umzusetzen. Dies machen wir in folgendem Beispiel.

In [47]:
public class PrintXMLVisitor implements Visitor {
    @Override
    public void visitDoor(Door d) {
        System.out.println("    <door></door>");
    }

    @Override
    public void visitWall(Wall w) {
        System.out.println("    <wall></wall>");
    }

    @Override
    public void visitRoom(Room r) {
        System.out.println("  <room>");
        for (MapSite mapSite : r.getChildren()) {
            mapSite.accept(this);
        }
        System.out.println("  </room>");
    }

    @Override
    public void visitMaze(Maze m) {
        System.out.println("<maze>");
        for (MapSite mapSite : m.getChildren()) {
            mapSite.accept(this);
        }
        System.out.println("</maze>");
    }

}

Nachdem die Traversierung der Kinderelemente nun im Visitor passiert, müssen wir den gleichen Code aus der Objektstruktur selbst entfernen.

In [48]:
class Room extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Room consisting of:");
        elements.forEach(x -> x.draw());
    }
    
    @Override
    public void accept(Visitor v) {
        v.visitRoom(this);
    }
}

In [49]:
class Maze extends Composite {
    @Override
    public void draw() {
        impl.updateGUI("Maze consisting of: ");
        elements.forEach(x -> x.draw());
    }
    
    @Override
    public void accept(Visitor v) {
        v.visitMaze(this);
    }
}

In [50]:
Maze maze = new Maze();
Room room1 = new Room();
Room room2 = new Room();
room1.add(new Wall());
room1.add(new Wall());
room1.add(new Wall());
room1.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room2.add(new Wall());
room1.add(new Door());
room2.add(new Door());

maze.add(room1);
maze.add(room2);

In [51]:
PrintXMLVisitor visitor = new PrintXMLVisitor()

In [52]:
maze.accept(visitor)

<maze>
  <room>
    <wall></wall>
    <wall></wall>
    <wall></wall>
    <wall></wall>
    <door></door>
  </room>
  <room>
    <wall></wall>
    <wall></wall>
    <wall></wall>
    <wall></wall>
    <door></door>
  </room>
</maze>


## Command

Das Command-Pattern erlaubt es uns, auszuführende Aktionen als Objekte darzustellen. Diese Objekte lassen sich herumreichen, und es ist möglich erst zu geeigneten Zeiten die Aktionen ausführen.

In [53]:
interface Command {
    void execute();
}

Wir erzeugen ein Dummy-Command für die Maze-Objektstruktur, das einfach nur aus einer Ausgabe besteht.

In [54]:
public class PrintCommand implements Command {
    private MapSite site;
    
    public PrintCommand(MapSite site) {
        this.site = site;
    }
    
    @Override
    public void execute() {
        System.out.println("Executing command on " + site.getClass().getSimpleName());
    }
}

In [55]:
public class OtherCommand implements Command {
    private MapSite site;
    
    public OtherCommand(MapSite site) {
        this.site = site;
    }

    @Override
    public void execute() {
        // site.otherStuff();
        // do other stuff

    }
}

Nun möchten wir ein Command für jedes Objekt innerhalb einer Objektstruktur einsammeln. Dafür eignet das Visitor Pattern.

In [56]:
public class CommandVisitor implements Visitor {

    private List<Command> commands = new ArrayList<>();

    public List<Command> getCommands() {
        return commands;
    }

    @Override
    public void visitDoor(Door d) {
        commands.add(new PrintCommand(d));
    }

    @Override
    public void visitWall(Wall w) {
        commands.add(new PrintCommand(w));
    }

    @Override
    public void visitRoom(Room r) {
        commands.add(new PrintCommand(r));
        r.getChildren().forEach(m -> m.accept(this));
    }

    @Override
    public void visitMaze(Maze m) {
        commands.add(new PrintCommand(m));
        m.getChildren().forEach(ms -> ms.accept(this));
    }
}

Nun können wir unsere Commands erzeugen, und nach belieben Ausführen.

In [57]:
CommandVisitor visitor = new CommandVisitor();
maze.accept(visitor);

In [58]:
List<Command> commands = visitor.getCommands();

In [59]:
commands.size()

13

In [60]:
for (Command command : visitor.getCommands()) {
    command.execute();
}

Executing command on Maze
Executing command on Room
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Door
Executing command on Room
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Door


## Strategy

Das Strategy-Muster wird verwendet, wenn zahlreiche zusammenhängende Klassen sich nur im Verhalten unterscheiden, oder verschiedene Varianten eines Algorithmus benötigt werden. Ein Beispiel für unser Maze-Szenario wären unterschiedliche Traversierungsstrategien, z.B. Preorder/Postorder. Das Strategy-Pattern erlaubt es, diese Strategien nach Belieben (und zur Laufzeit) zu wechseln, und auch einfach neue Strategien einzubauen.

In [61]:
interface TraversalStrategy {
    List<MapSite> traverse(MapSite element);
}

In [62]:
public class PreorderTraversalStrategy implements TraversalStrategy {
    public List<MapSite> traverse(MapSite element) {
        List<MapSite> elements = new ArrayList<MapSite>();
        elements.add(element);
        if (element instanceof Composite) {
            for (MapSite map : ((Composite)element).getChildren()) {
                elements.addAll(traverse(map));
            }
        }
        return elements;
    }
}

In [63]:
public class PostorderTraversalStrategy implements TraversalStrategy {
    public List<MapSite> traverse(MapSite element) {
        List<MapSite> elements = new ArrayList<MapSite>();
        if (element instanceof Composite) {
            for (MapSite map : ((Composite)element).getChildren()) {
                elements.addAll(traverse(map));
            }
        }
        elements.add(element);

        return elements;
    }
}

In [64]:
TraversalStrategy strategy = new PreorderTraversalStrategy();
List<MapSite> preorderElements = strategy.traverse(maze)

In [65]:
for (MapSite element : preorderElements) {
    new PrintCommand(element).execute();
}

Executing command on Maze
Executing command on Room
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Door
Executing command on Room
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Door


In [66]:
TraversalStrategy strategy = new PostorderTraversalStrategy();
List<MapSite> postorderElements = strategy.traverse(maze);

In [67]:
for (MapSite element : postorderElements) {
    new PrintCommand(element).execute();
}

Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Door
Executing command on Room
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Wall
Executing command on Door
Executing command on Room
Executing command on Maze
